Capstone 1 

All companies that participate in wholesale energy markets are required to submit a quarterly report detailing each transaction to the Federal Energy Regulatory Commission (FERC). This information is then made publicly available for download on FERC’s website at https://eqrreportviewer.ferc.gov/.

I've dowloaded all of the hourly data for Florida as a csv file and saved it to my desktop. My first step is to load the file as a dataframe.

In [53]:
import pandas as pd
from sklearn import *
from math import sqrt
import numpy as np

data = pd.read_csv(r'C:\Users\anhem44\Desktop\Capstone1\all_data_cap1.csv') #import data
data.head()

,DATE,SELLER_COMPANY,SELLER_COMPANY_OLD,C_BUYER_NAME,C_BUYER_NAME_OLD,Region,Contract_Service_Agreement_id,TR_CONTRACT_ID,loc,TR_TIMEZONE,...,index_loc_seller,index_loc_oldseller,transcharge,price_minus_index,price_above_trans,price_above_trans_mwh,transaction_len,year,qtr,benchhub
0,1/1/2014,EXELON,"EXELON GENERATION COMPANY, LLC",THE ENERGY AUTHORITY,THE ENERGY AUTHORITY,Florida,12870,717562,FPL,EASTERNPREVAILING,...,30.000000,30.000000,8,-1.471054,0,0.0,Hourly,2014,1,FPL
1,1/1/2014,SOUTHERN COMPANY,"SOUTHERN COMPANY SERVICES, INC. (AS AGENT)",SEMINOLE ELECTRIC COOP,"SEMINOLE ELECTRIC COOPERATIVE, INC.",Florida,481,1888312,FPL,CENTRALSTANDARD,...,38.000000,38.000000,8,2.114532,0,0.0,Hourly,2014,1,FPL
2,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",J.P. MORGAN CHASE & COMPANY,JP MORGAN VENTURES ENERGY CORPORATION,Florida,30760,892022,FPL,EASTERNPREVAILING,...,31.750000,31.750000,8,0.893512,0,0.0,Hourly,2014,1,FPL
3,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",EXELON,"EXELON GENERATION COMPANY, LLC",Florida,30789,892011,FPL,EASTERNPREVAILING,...,31.500000,31.500000,8,1.272839,0,0.0,Hourly,2014,1,FPL
4,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",EXELON,"EXELON GENERATION COMPANY, LLC",Florida,30789,892011,FPL,EASTERNPREVAILING,...,31.451613,31.451613,8,1.407159,0,0.0,Hourly,2014,1,FPL


With the data loaded I double check the datatypoes to isnure that there were no errors in the upload.

In [54]:
data.dtypes #check types

DATE                              object
SELLER_COMPANY                    object
SELLER_COMPANY_OLD                object
C_BUYER_NAME                      object
C_BUYER_NAME_OLD                  object
Region                            object
Contract_Service_Agreement_id      int64
TR_CONTRACT_ID                     int64
loc                               object
TR_TIMEZONE                       object
TR_CLASS_NAME                     object
PRICEINDOLPERMWH                 float64
TR_DELV_SPEC_LOC                  object
TRADE_DATE                        object
HOUROFDAY                          int64
QUANTITYINMWH                    float64
HOURLYTRANSCHARGE                float64
HOUR_FREQ                          int64
weighted_pricemw                 float64
index_loc                        float64
index_bench                      float64
index_loc_seller                 float64
index_loc_oldseller              float64
transcharge                        int64
price_minus_inde

Everything looks good, with the exception of the 'TRADE_DATE' column. This column contains the date and hour that the trade occured and should be a date time, but during upload it was classified as an object. So my next step is to convert it to a data time and check the results.

In [55]:
data['TRADE_DATE'] = pd.to_datetime(data['TRADE_DATE']) #convert to DT
data['TRADE_DATE'].head()

0   2014-01-01 22:00:00
1   2014-01-01 18:00:00
2   2014-01-01 02:00:00
3   2014-01-01 03:00:00
4   2014-01-01 04:00:00
Name: TRADE_DATE, dtype: datetime64[ns]

Since the goal of this project was to predict prices for a FERC specific time period and season I used the following to create these variables in the data frame.

In [56]:
data['FERC_time'] = 0 #'off_peak' #make everthing off peak and then change it to peak if criteria are met
data['FERC_time'][(data['TRADE_DATE'].dt.weekday <= 5) & (data['TRADE_DATE'].dt.hour >= 6) & (data['TRADE_DATE'].dt.hour <= 21)]=1 #'peak' 

data['FERC_season'] = 0 #'shoulder' #make everthing shoulder and then change it to winter or summer if criteria are met
data['FERC_season'][(data['TRADE_DATE'].dt.month <= 2) | (data['TRADE_DATE'].dt.month >= 12)]= 1 # winter' 
data['FERC_season'][(data['TRADE_DATE'].dt.month <= 8) & (data['TRADE_DATE'].dt.month >= 6)]= 2 #'summer'
data.head()

C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,DATE,SELLER_COMPANY,SELLER_COMPANY_OLD,C_BUYER_NAME,C_BUYER_NAME_OLD,Region,Contract_Service_Agreement_id,TR_CONTRACT_ID,loc,TR_TIMEZONE,...,transcharge,price_minus_index,price_above_trans,price_above_trans_mwh,transaction_len,year,qtr,benchhub,FERC_time,FERC_season
0,1/1/2014,EXELON,"EXELON GENERATION COMPANY, LLC",THE ENERGY AUTHORITY,THE ENERGY AUTHORITY,Florida,12870,717562,FPL,EASTERNPREVAILING,...,8,-1.471054,0,0.0,Hourly,2014,1,FPL,0,1
1,1/1/2014,SOUTHERN COMPANY,"SOUTHERN COMPANY SERVICES, INC. (AS AGENT)",SEMINOLE ELECTRIC COOP,"SEMINOLE ELECTRIC COOPERATIVE, INC.",Florida,481,1888312,FPL,CENTRALSTANDARD,...,8,2.114532,0,0.0,Hourly,2014,1,FPL,1,1
2,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",J.P. MORGAN CHASE & COMPANY,JP MORGAN VENTURES ENERGY CORPORATION,Florida,30760,892022,FPL,EASTERNPREVAILING,...,8,0.893512,0,0.0,Hourly,2014,1,FPL,0,1
3,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",EXELON,"EXELON GENERATION COMPANY, LLC",Florida,30789,892011,FPL,EASTERNPREVAILING,...,8,1.272839,0,0.0,Hourly,2014,1,FPL,0,1
4,1/1/2014,THE ENERGY AUTHORITY,"THE ENERGY AUTHORITY, INC.",EXELON,"EXELON GENERATION COMPANY, LLC",Florida,30789,892011,FPL,EASTERNPREVAILING,...,8,1.407159,0,0.0,Hourly,2014,1,FPL,0,1


Based on the conclusions I reached when exploring the data (see data story) I need variables for all of the following:

1.	Time
    *	Hour
    *	Day
2.	FERC season
3.	FERC period
4.	Frequency of trading
5.	Location
6.	Entity
    *	Entity as seller
    *	Entity as purchaser 

In [57]:
data['is_sunday'] = data['TRADE_DATE'].dt.weekday == 6 #extract sunday
data['is_saturday'] = data['TRADE_DATE'].dt.weekday == 5 #extract saturday
data['is_weekday'] = data['TRADE_DATE'].dt.weekday <= 5 #extract weekday

data_y = data.copy() #copy the data

#subset data to predictive factors
data_x = data[['FERC_time','FERC_season','is_sunday','is_saturday','is_weekday','HOUR_FREQ',
               'HOUROFDAY','SELLER_COMPANY','C_BUYER_NAME','TR_DELV_SPEC_LOC']]

#get dummies for categorical data and check
data_x = pd.get_dummies(data_x)
data_x.head()

,FERC_time,FERC_season,is_sunday,is_saturday,is_weekday,HOUR_FREQ,HOUROFDAY,SELLER_COMPANY_CARGILL,SELLER_COMPANY_EXELON,SELLER_COMPANY_MORGAN STANLEY,...,TR_DELV_SPEC_LOC_POU,TR_DELV_SPEC_LOC_SC,TR_DELV_SPEC_LOC_SCEG,TR_DELV_SPEC_LOC_SOCO,TR_DELV_SPEC_LOC_SOCOLOAD,TR_DELV_SPEC_LOC_SSN,TR_DELV_SPEC_LOC_TAL,TR_DELV_SPEC_LOC_TEC,TR_DELV_SPEC_LOC_TEC/FPL,TR_DELV_SPEC_LOC_TREDWINDS SUBSTATION
0,0,1,False,False,True,1,22,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,False,False,True,1,18,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,False,False,True,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,False,False,True,3,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,False,False,True,3,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that I have my data the next step is to strip out the price as a seperate variable and split the data between a training and testing set.

In [58]:
#split data into traning and testing set
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(data_x, data_y.PRICEINDOLPERMWH, random_state = 1)

Now that we have our training and testing data we run the regression and train our model

In [59]:
#use SGDRegressor w/200 iterations and fit training data
regr= linear_model.SGDRegressor(n_iter=200, verbose=3)
regr = regr.fit(X_train,Y_train)

-- Epoch 1
Norm: 16.93, NNZs: 87, Bias: 13.520630, T: 59337, Avg. loss: 38.824930
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 19.03, NNZs: 87, Bias: 14.496144, T: 118674, Avg. loss: 36.576659
Total training time: 0.05 seconds.
-- Epoch 3
Norm: 20.40, NNZs: 87, Bias: 14.963755, T: 178011, Avg. loss: 35.609127
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 21.57, NNZs: 87, Bias: 15.205098, T: 237348, Avg. loss: 35.021984
Total training time: 0.10 seconds.
-- Epoch 5
Norm: 22.67, NNZs: 87, Bias: 15.450911, T: 296685, Avg. loss: 34.636457
Total training time: 0.13 seconds.
-- Epoch 6
Norm: 23.55, NNZs: 87, Bias: 15.536551, T: 356022, Avg. loss: 34.348584
Total training time: 0.15 seconds.
-- Epoch 7
Norm: 24.36, NNZs: 87, Bias: 15.733266, T: 415359, Avg. loss: 34.121699
Total training time: 0.18 seconds.
-- Epoch 8
Norm: 25.06, NNZs: 87, Bias: 15.835279, T: 474696, Avg. loss: 33.937562
Total training time: 0.21 seconds.
-- Epoch 9
Norm: 25.81, NNZs: 87, Bias: 15.965646, T: 534

Next use our test set to see how well the model did by computing the root mean square error.

In [60]:
#do prediction with testing data and compute mean square error
Y_pred = regr.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(Y_test, Y_pred))
rmse 

7.793163121866575

We compute our 90% confidence interval.

In [61]:
#compute how much more or less the price will be
con_int = (1.645 * (Y_pred.std()/sqrt(len(Y_pred))))

print("+/-:",con_int)

+/-: 0.0780525895376


According to the above our the predicted price is +/- $.06 of the actual price.

We can check for overfitting by checking the mean squared error of both the training and testing set.

In [62]:
Y_pred_train = regr.predict(X_train) #use training set to predict

mse_train = metrics.mean_squared_error(Y_pred_train, Y_train) #training error
mse_test = metrics.mean_squared_error(Y_pred, Y_test) #testing error

print("training error: ",mse_train)
print("testing error: ",mse_test)

training error:  61.8092074961
testing error:  60.733391444


Our errors are very similar suggesting a good fit.

Next we will cross validate the data using the kfolds method.

In [63]:
kfold = cross_validation.KFold(len(data_x),n_folds=5, random_state=1)
r_pred_error = [] #list to record errors

for train, test in kfold:
    kx_train, kx_test, ky_train, ky_test = data_x.iloc[train], data_x.iloc[test], data_y.PRICEINDOLPERMWH[train], data_y.PRICEINDOLPERMWH[test]
    regr_k = linear_model.SGDRegressor()
    regr_k.fit(kx_train, ky_train)
    pred = regr_k.predict(kx_test)
    r_pred_error.append(sqrt(metrics.mean_squared_error(pred, ky_test)))

print(r_pred_error)
print(np.mean(r_pred_error))

[11.855956000290726, 9.874174201351895, 4.802114011719877, 7.4254032019799405, 10.376600218878359]
8.86684952684


The mean of our errors seems to vary somewhat, however, the mean is somewhat higher. 

Finally, since the goal of this project was to predict the prices in each of the predifiened periods and seasons we will compute the actual averages with the predicted averages and compare results.

In [64]:
# Legend:
    # FERC Season
        # 0 = off_peak
        # 1 = peak
    # FERC Time
        # 0 = Shoulder
        # 1 = Winter
        # 2 = Summer

def compare_season_price(season, time):
    season_name = data_y.loc[data_y['FERC_season'] == season] #subset y data
    period = season_name.loc[season_name['FERC_time'] == time]
    
    real_prices = period.PRICEINDOLPERMWH #avg actual prices
    average_price = real_prices.mean()
    
    season_x = data_x.loc[data_x['FERC_season'] == season] #subset x data
    period_x = season_x.loc[season_x['FERC_time'] == time]
    
    csp_pred = regr.predict(period_x) #predict future prices
    
    pred_price = csp_pred.mean() #avg prdicted prices
    rmse = sqrt(metrics.mean_squared_error(csp_pred, real_prices)) #mean square error
    csp_con_int = (1.645 * (csp_pred.std()/sqrt(len(csp_pred)))) #confidence interval
    
    return print("RMSE:",rmse), print("confidence +/-",csp_con_int),print("actual price:",average_price),print("pred price:",pred_price)

In [65]:
#Shoulder off-peak
compare_season_price(0,0)

RMSE: 6.460995165023053
confidence +/- 0.0717961830879
actual price: 21.0122451050622
pred price: 21.0206847355


(None, None, None, None)

In [66]:
#Shoulder peak
compare_season_price(0,1)

RMSE: 8.046754590578736
confidence +/- 0.0725068619786
actual price: 25.02156445864577
pred price: 25.1449962946


(None, None, None, None)

In [67]:
#Winter off-peak
compare_season_price(1,0)

RMSE: 7.172480775404076
confidence +/- 0.0974984013629
actual price: 19.88418845315916
pred price: 21.7049661425


(None, None, None, None)

In [68]:
#Winter peak
compare_season_price(1,1)

RMSE: 7.995408040775986
confidence +/- 0.0767670832464
actual price: 21.24755833581574
pred price: 24.6237847905


(None, None, None, None)

In [69]:
#Summer off-peak
compare_season_price(2,0)

RMSE: 6.269108183407658
confidence +/- 0.103263367056
actual price: 22.869034327879284
pred price: 23.2016712853


(None, None, None, None)

In [70]:
#Summer peak
compare_season_price(2,1)

RMSE: 10.355819986776966
confidence +/- 0.122235675441
actual price: 28.402529495887926
pred price: 27.8207942347


(None, None, None, None)